# Running on Colab, unzipping augmented Data from drive.

In [1]:
!unzip /content/drive/MyDrive/midas/trainPart1.zip

Streaming output truncated to the last 5000 lines.
  inflating: trainPart1/train/f/27.jpg  
  inflating: trainPart1/train/f/28.jpg  
  inflating: trainPart1/train/f/29.jpg  
  inflating: trainPart1/train/f/30.jpg  
  inflating: trainPart1/train/f/31.jpg  
  inflating: trainPart1/train/f/32.jpg  
  inflating: trainPart1/train/f/33.jpg  
  inflating: trainPart1/train/f/34.jpg  
  inflating: trainPart1/train/f/35.jpg  
  inflating: trainPart1/train/f/36.jpg  
  inflating: trainPart1/train/f/37.jpg  
  inflating: trainPart1/train/f/38.jpg  
  inflating: trainPart1/train/f/39.jpg  
  inflating: trainPart1/train/f/40.jpg  
  inflating: trainPart1/train/f/41.jpg  
  inflating: trainPart1/train/f/42.jpg  
  inflating: trainPart1/train/f/43.jpg  
  inflating: trainPart1/train/f/44.jpg  
  inflating: trainPart1/train/f/45.jpg  
  inflating: trainPart1/train/f/46.jpg  
  inflating: trainPart1/train/f/47.jpg  
  inflating: trainPart1/train/f/48.jpg  
  inflating: trainPart1/train/f/49.jpg  
  infl

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import LeakyReLU

import keras
import tensorflow as tf

# 52 classes of alphabets with A-Z, a-z and 10 classes of numbers 0-9

In [3]:
!ls ./trainPart1/train/ | wc -l

62


Setting ImageDataGenerator with the required augmentation properties. ref : https://www.kaggle.com/jeffreypaul15/introduction-to-cnn-keras-0-997-top-6/edit
And splitting them according to training and validation with 20 % split


In [4]:
directory = '/content/trainPart1/train'

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1, 
        validation_split=0.2# randomly shift images vertically (fraction of total height)
        )

tds = datagen.flow_from_directory(
    directory=directory,
    target_size=(100, 100),
    color_mode="grayscale",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset="training",
    interpolation="nearest",
)

testds = datagen.flow_from_directory(
    directory=directory,
    target_size=(100, 100),
    color_mode="grayscale",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset="validation",
    interpolation="nearest",
)

Found 5943 images belonging to 62 classes.
Found 1483 images belonging to 62 classes.


# Creating a CNN architecture with CNN -> Pooling -> CNN -> Pooling -> CNN -> Pooling -> CNN -> Pooling -> CNN -> Pooling -> CNN -> Pooling -> Flatten -> 2 hidden layers and an output layer

In [5]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu', input_shape = (100, 100, 1)))
model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))


model.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 512, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same'))
model.add(LeakyReLU(alpha=0.3))

# model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(128, activation = "relu"))
model.add(Dense(62, activation = "softmax"))

In [6]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="/content/drive/MyDrive/midas/model_check/62class.hdf5",
                                                 save_weights_only=True,
                                                 verbose=1)


history = model.fit(tds, epochs = 20, validation_data = testds, batch_size = 64,
                    verbose = 1, callbacks=[cp_callback])


Epoch 1/20
186/186 [==============================] - 163s 703ms/step - loss: 4.2632 - accuracy: 0.0134 - val_loss: 4.1274 - val_accuracy: 0.0148

Epoch 00001: saving model to /content/drive/MyDrive/midas/model_check/62class.hdf5
Epoch 2/20
186/186 [==============================] - 127s 685ms/step - loss: 4.1321 - accuracy: 0.0174 - val_loss: 4.1271 - val_accuracy: 0.0135

Epoch 00002: saving model to /content/drive/MyDrive/midas/model_check/62class.hdf5
Epoch 3/20
186/186 [==============================] - 127s 683ms/step - loss: 4.1316 - accuracy: 0.0182 - val_loss: 4.1256 - val_accuracy: 0.0155

Epoch 00003: saving model to /content/drive/MyDrive/midas/model_check/62class.hdf5
Epoch 4/20
186/186 [==============================] - 128s 686ms/step - loss: 4.1294 - accuracy: 0.0160 - val_loss: 4.1251 - val_accuracy: 0.0162

Epoch 00004: saving model to /content/drive/MyDrive/midas/model_check/62class.hdf5
Epoch 5/20
186/186 [==============================] - 127s 684ms/step - loss: 4.